# Extra

In [2]:
def resizeImage(img_path, new_width):
    img = cv2.imread(img_path)
    height, width = img.shape[:2]
    ratio = float(new_width) / width
    new_height = int(height * ratio)
    resized_img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_AREA)

    return resized_img

In [49]:
# Import libraries
from plantcv import plantcv as pcv 
import warnings

class options:
    def __init__(self):
        #self.image = '../../leafsnap/leafsnap-dataset/dataset/images/field/acer_rubrum/13001155906945.jpg'
        # self.image = '../images/raws/acer_rubrum/13001155906945.jpg'
        self.debug = "none"
        self.writeimg= 'False' 
        self.result = "features_metadata.json"
        self.outdir = ""
        self.verbose= False
# Get options
args = options()

# Set debug to the global parameter 
pcv.params.debug = args.debug
pcv.params.debug = args.debug


def preprocess_image(image_path):
    # img, path, filename = pcv.readimage(filename=image_path)
    img = resizeImage(image_path, 300)
    # Convert RGB to HSV and extract the saturation channel
    s = pcv.rgb2gray_hsv(rgb_img=img, channel='s')
    # Threshold the saturation image
    s_thresh = pcv.threshold.binary(gray_img=s, threshold=85, max_value=255, object_type='light')
    # Median Blur
    s_mblur = pcv.median_blur(gray_img=s_thresh, ksize=5)
    s_cnt = pcv.median_blur(gray_img=s_thresh, ksize=5)
    # Convert RGB to LAB and extract the Blue channel
    #b = pcv.rgb2gray_lab(gray_img=img, channel='b')
    b = pcv.rgb2gray_lab(rgb_img=img, channel='b')
    # Threshold the blue image
    b_thresh = pcv.threshold.binary(gray_img=b, threshold=160, max_value=255, object_type='light')
    b_cnt = pcv.threshold.binary(gray_img=b, threshold=160, max_value=255, object_type='light')
    # Join the thresholded saturation and blue-yellow images
    bs = pcv.logical_or(bin_img1=s_mblur, bin_img2=b_cnt)
    # Apply Mask (for VIS images, mask_color=white)
    #masked = pcv.apply_mask(rgb_img=img, mask=bs, mask_color='white')
    masked = pcv.apply_mask(img=img, mask=bs, mask_color='white')
    # Convert RGB to LAB and extract the Green-Magenta and Blue-Yellow channels
    masked_a = pcv.rgb2gray_lab(rgb_img=masked, channel='a')
    masked_b = pcv.rgb2gray_lab(rgb_img=masked, channel='b')
    # Threshold the green-magenta and blue images
    maskeda_thresh = pcv.threshold.binary(gray_img=masked_a, threshold=115, max_value=255, object_type='dark')
    maskeda_thresh1 = pcv.threshold.binary(gray_img=masked_a, threshold=135, max_value=255, object_type='light')
    maskedb_thresh = pcv.threshold.binary(gray_img=masked_b, threshold=128, max_value=255, object_type='light')
    # Join the thresholded saturation and blue-yellow images (OR)
    ab1 = pcv.logical_or(bin_img1=maskeda_thresh, bin_img2=maskedb_thresh)
    ab = pcv.logical_or(bin_img1=maskeda_thresh1, bin_img2=ab1)
    # Fill small objects
    ab_fill = pcv.fill(bin_img=ab, size=200)
    fill_image = pcv.fill_holes(bin_img=ab)
    # Apply mask (for VIS images, mask_color=white)
    #masked2 = pcv.apply_mask(rgb_img=masked, mask=ab_fill, mask_color='white')
    masked2 = pcv.apply_mask(img=masked, mask=fill_image, mask_color='white')
    skeleton = pcv.morphology.skeletonize(mask=masked2)
    # Identify objects
    id_objects, obj_hierarchy = pcv.find_objects(img=img, mask=fill_image)

    # get dimensions of image
    dimensions = img.shape
    # height, width, number of channels in image
    # height = img.shape[0] - (img.shape[0] * (30 / 100))
    # width = img.shape[1] - (img.shape[1] * (30 / 100))

    middle_x = img.shape[1] // 2
    middle_y = img.shape[0] // 2

    # Define ROI
    # roi1, roi_hierarchy= pcv.roi.circle(img=masked2, x=50, y=50, h=height, w=width)
    roi1, roi_hierarchy= pcv.roi.circle(img=masked2, x=middle_x, y=middle_y, r=50)

    warnings.filterwarnings('ignore')
    
    # Decide which objects to keep
    roi_objects, hierarchy3, kept_mask, obj_area = pcv.roi_objects(img=img, roi_contour=roi1, 
                                                                    roi_hierarchy=roi_hierarchy, 
                                                                    object_contour=id_objects, 
                                                                    obj_hierarchy=obj_hierarchy,
                                                                    roi_type='largest')
                                                                
    # Object combine kept objects
    obj, mask = pcv.object_composition(img=img, contours=roi_objects, hierarchy=hierarchy3)

    # Return contours, thresh/binary, gray
    return [roi_objects, kept_mask, img]

# HoCS-Related

In [129]:
import numpy as np
from scipy.spatial import distance


def calculate_area_measure(contour_points, radii):
    x = []
    y = []
    x_y = []
    for cp in contour_points[0]:
        x.append(cp[0][0])
        y.append(cp[0][1])
        x_y.append([cp[0][0], cp[0][1]])

    centroid = np.mean(x_y, axis=0)
    abs_centroid = [int(centroid[0]), int(centroid[1])]
    area_measures = []

    for radius in radii:
        area = 0.0
        for cp in x_y:
            dist = distance.euclidean(cp, abs_centroid)
            if dist <= radius:
                area += 1.0
        area_measures.append(area)
    return area_measures


def calculate_curvature(contour_points):
    x = []
    y = []
    x_y = []
    for cp in contour_points[0]:
        x.append(cp[0][0])
        y.append(cp[0][1])
        x_y.append([cp[0][0], cp[0][1]])

        
    curvatures = []
    for i in range(len(x_y) - 2):
        x1, y1 = x_y[i]
        x2, y2 = x_y[i+1]
        x3, y3 = x_y[i+2]
        curvature = abs((x1-x2)*(y2-y3) - (x2-x3)*(y1-y2)) / ((x1-x2)**2 + (y1-y2)**2)**1.5
        curvatures.append(curvature)
    return curvatures


def generate_histogram(area_measure, curvature_values, bins, range):
    weights = np.repeat(area_measure, len(curvature_values))
    histogram, _ = np.histogram(curvature_values, bins=bins, range=range, weights=weights)
    return histogram

In [134]:
import os
import cv2
import pandas as pd

path = "../compiled/"
counter = 0

# initialize empty lists for features and labels
hocs_list = []
labels = []

error_num = 0

# loop through all subfolders in the path
for foldername in os.listdir(path):
    folderpath = os.path.join(path, foldername)
    if not os.path.isdir(folderpath):
        continue
    # loop through all image files in the subfolder
    for filename in os.listdir(folderpath):

        # For testing to limit outputs
        if counter == 5: break

        filepath = os.path.join(folderpath, filename)
        if not os.path.isfile(filepath):
            continue

        try:
            pcv_contour = preprocess_image(filepath)
            contours = pcv_contour[0]
            thresh = pcv_contour[1]
            image = pcv_contour[2]

            curvatures = calculate_curvature(contours)
            histogram, bins = np.histogram(curvatures, bins=25, range=(0.0, 1.0))
            flattened_hist = histogram.flatten()

            hocs_list.append(flattened_hist)
            labels.append(foldername)

            counter += 1
        except:
            error_num += 1
            counter += 1
            print(error_num)
            pass

    print(f"Finished plant: {foldername}")


# save features and labels to a pandas dataframe and export to CSV file
data = pd.DataFrame({"hocs": hocs_list})
data["plant_name"] = labels
data.to_csv("../csv/HoCS.csv", index=False)

Finished plant: acer_campestre
Finished plant: acer_ginnala
Finished plant: acer_pensylvanicum
Finished plant: acer_rubrum
Finished plant: amelanchier_canadensis
Finished plant: asimina_triloba
Finished plant: betula_alleghaniensis
Finished plant: betula_nigra
Finished plant: betula_populifolia
Finished plant: castanea_dentata
Finished plant: catalpa_bignonioides
Finished plant: crataegus_pruinosa
Finished plant: fagus_grandifolia
Finished plant: liriodendron_tulipifera
Finished plant: populus_deltoides
Finished plant: populus_grandidentata
Finished plant: quercus_bicolor
Finished plant: quercus_nigra
Finished plant: styrax_japonica
Finished plant: ulmus_rubra
Finished plant: vitex_negundo
Finished plant: zelkova_serrata
